---
title: STEM post processing
authors: [Toma Susi]
date: 2024/07/24
---

In [2]:
%matplotlib ipympl
import abtem


abtem.config.set({"visualize.cmap": "viridis"})
abtem.config.set({"visualize.continuous_update": True})
abtem.config.set({"visualize.autoscale": True})

all_measurements = abtem.from_zarr("data/processed_STEM.zarr")

In [12]:
#| label: app:stem_processing
viz = all_measurements.show(interact=True, explode=(1,), figsize=(5,5))

[########################################] | 100% Completed | 103.55 ms


ImageGUI(children=(VBox(children=(SelectionSlider(options=('Nyquist-sampled', 'Interpolated', 'Blurred', 'Nois…